In [1]:
import pandas as pd

import mysql.connector
from datetime import datetime, timedelta


In [2]:
conn = mysql.connector.connect(host='10.90.21.39', port='3339', username='thuypt8', password='dmJ2sP9xTTU62sZq', database='um')
my_cursor = conn.cursor()
query = "SELECT * FROM loyalty_active_user LIMIT 2000000"
my_cursor.execute(query)
rows = my_cursor.fetchall()
columns = [i[0] for i in my_cursor.description]
dataSQL = pd.DataFrame(rows, columns=columns)

In [3]:
conn.commit()
conn.close()

print("Connection succesfully created!")

Connection succesfully created!


In [4]:
df = dataSQL


# Tính RR30

Tính log_date (lần đăng nhập gần nhất) và first_date (lần đăng nhập đầu tiên) của người dùng

In [5]:
new_df1 = df.groupby(['loyalty_uid','log_date']).size().reset_index()

new_df1 = new_df1.rename(columns={'log_date': 'login_date'}).drop(0, axis=1)
# print(new_df1)

new_df2 = df.groupby('loyalty_uid').agg({
    'log_date': 'min'
})
new_df2 = new_df2.rename(columns={'log_date': 'first_date'})
# print(new_df2)

Gộp thông tin login_date và first_date theo loyalty_uid


In [6]:
new_merged_df = pd.merge(new_df2, new_df1, how='inner',on='loyalty_uid')

# Chuyển đổi dữ liệu thành datetime để tránh lỗi
new_merged_df['login_date'] = pd.to_datetime(new_merged_df['login_date'])
new_merged_df['first_date'] = pd.to_datetime(new_merged_df['first_date'])

# Tính khoảng cách từ lần đầu đến lần cuối đăng nhập
new_merged_df['day_number'] = (new_merged_df['login_date'] - new_merged_df['first_date'])

# print(new_merged_df)

Tính lượng người dùng truy cập theo ngày đầu truy cập và khoảng cách từ ngày đầu đến ngày cuối truy cập

In [7]:
data_day = new_merged_df.groupby(['first_date', 'day_number']).size()
data_day = data_day.reset_index()
# print(data_day)

In [8]:
def calculate_retention_rate_date(df, datetime_str, n_day):
    datetime_value = pd.to_datetime(datetime_str)
    new_merged_df['login_date'] = pd.to_datetime(new_merged_df['login_date'])
    new_merged_df['first_date'] = pd.to_datetime(new_merged_df['first_date'])
    retention_rate = []
    n_day = 31 - n_day
    for date_number in range(0, n_day):
        end_of_date = datetime_value + timedelta(days=date_number)
        # print(datetime_value, end_of_month)

        new_users = set(df[(df['login_date'] == end_of_date)]['loyalty_uid'])

        first_date_users = set(df[df['first_date'] == datetime_value]['loyalty_uid'])

        value = len(new_users.intersection(first_date_users)) / len(first_date_users) * 100

        retention_rate.insert(date_number, value)
        # retention_rate[date_number] = len(new_users.intersection(first_date_users)) / len(first_date_users) * 100

    return retention_rate

Nhập tay dữ liệu ngày

In [9]:
date_input = input('Nhập ngày cần tính: ') # 2023-10-20
date_inp = pd.to_datetime(date_input)

In [10]:
# Test function và xem kết quả
retention_rate_result10 = calculate_retention_rate_date(new_merged_df, date_inp, 0)
print(retention_rate_result10)

[100.0, 5.321507760532151, 4.138950480413895, 3.473762010347376, 3.917220990391722, 2.5868440502586845, 2.5129342202512936, 3.0303030303030303, 2.5129342202512936, 2.7346637102734666, 2.3651145602365116, 2.8085735402808574, 2.2912047302291203, 2.7346637102734666, 2.3651145602365116, 1.8477457501847747, 0.8130081300813009, 0.5912786400591279, 2.5868440502586845, 1.7738359201773837, 1.8477457501847747, 1.6260162601626018, 1.6999260901699924, 1.9216555801921658, 1.4781966001478197, 1.6260162601626018, 3.0303030303030303, 2.0694752402069474, 1.5521064301552108, 3.0303030303030303, 1.8477457501847747]


In [16]:
cohort = []


for i in range(0, 31):
    p_date = date_inp + timedelta(days=i)
    retention_rate_result_new = calculate_retention_rate_date(new_merged_df, p_date, i)
    cohort.insert(i, retention_rate_result_new)

In [12]:
date_arr = []
for i in range(0, 31):
    p_date = date_inp + timedelta(days=i)
    date_arr.insert(i,p_date)

new_data10 = pd.DataFrame()
new_data10 = date_arr


In [13]:
print(new_data10)

[Timestamp('2023-10-01 00:00:00'), Timestamp('2023-10-02 00:00:00'), Timestamp('2023-10-03 00:00:00'), Timestamp('2023-10-04 00:00:00'), Timestamp('2023-10-05 00:00:00'), Timestamp('2023-10-06 00:00:00'), Timestamp('2023-10-07 00:00:00'), Timestamp('2023-10-08 00:00:00'), Timestamp('2023-10-09 00:00:00'), Timestamp('2023-10-10 00:00:00'), Timestamp('2023-10-11 00:00:00'), Timestamp('2023-10-12 00:00:00'), Timestamp('2023-10-13 00:00:00'), Timestamp('2023-10-14 00:00:00'), Timestamp('2023-10-15 00:00:00'), Timestamp('2023-10-16 00:00:00'), Timestamp('2023-10-17 00:00:00'), Timestamp('2023-10-18 00:00:00'), Timestamp('2023-10-19 00:00:00'), Timestamp('2023-10-20 00:00:00'), Timestamp('2023-10-21 00:00:00'), Timestamp('2023-10-22 00:00:00'), Timestamp('2023-10-23 00:00:00'), Timestamp('2023-10-24 00:00:00'), Timestamp('2023-10-25 00:00:00'), Timestamp('2023-10-26 00:00:00'), Timestamp('2023-10-27 00:00:00'), Timestamp('2023-10-28 00:00:00'), Timestamp('2023-10-29 00:00:00'), Timestamp('20

In [17]:
for i in range(0,31):
    cohort[i].insert(0, new_data10[i])

In [18]:
# in thử thông tin cohort
for i in range(0,31):
    print(cohort[i])
result_df = pd.DataFrame(cohort)

[Timestamp('2023-10-01 00:00:00'), 100.0, 5.321507760532151, 4.138950480413895, 3.473762010347376, 3.917220990391722, 2.5868440502586845, 2.5129342202512936, 3.0303030303030303, 2.5129342202512936, 2.7346637102734666, 2.3651145602365116, 2.8085735402808574, 2.2912047302291203, 2.7346637102734666, 2.3651145602365116, 1.8477457501847747, 0.8130081300813009, 0.5912786400591279, 2.5868440502586845, 1.7738359201773837, 1.8477457501847747, 1.6260162601626018, 1.6999260901699924, 1.9216555801921658, 1.4781966001478197, 1.6260162601626018, 3.0303030303030303, 2.0694752402069474, 1.5521064301552108, 3.0303030303030303, 1.8477457501847747]
[Timestamp('2023-10-02 00:00:00'), 100.0, 7.373271889400922, 5.0691244239631335, 8.847926267281107, 5.345622119815668, 5.161290322580645, 6.5437788018433185, 6.451612903225806, 3.5023041474654377, 5.0691244239631335, 4.516129032258064, 3.225806451612903, 4.7926267281106, 3.317972350230415, 2.5806451612903225, 1.6589861751152075, 1.3824884792626728, 5.437788018

In [19]:
result_df.to_clipboard()